In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from finta import TA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.regularizers import l2
from sklearn.model_selection import KFold

In [2]:
# Load data
AMZN_path = Path("../files/AMZN.csv")
VIX_path = Path("../files/^VIX.csv")
FEDFUNDS_path = Path("../files/FEDFUNDS (1).csv")
SPY_index = Path("../files/SPY.csv")

In [3]:
amzn_df = pd.read_csv(AMZN_path, index_col="Date")
fear_index_df = pd.read_csv(VIX_path, index_col="Date").rename(columns={"Close": "Fear_index"})
spy_index_df = pd.read_csv(SPY_index, index_col="Date").rename(columns={"Close": "SPY_index"})
fedfunds_df = pd.read_csv(FEDFUNDS_path, index_col="DATE")

In [4]:
# Convert index to datetime index
fedfunds_df.index = pd.to_datetime(fedfunds_df.index)

# Resample the fed_funds_df to have daily frequency and forward fill the values
fedfunds_df_monthly = fedfunds_df.resample('D').ffill()


In [5]:
# Concatenate dataframes
concatenated_df = pd.concat([amzn_df, fear_index_df['Fear_index'], spy_index_df['SPY_index']], axis=1)
concatenated_df.index = pd.to_datetime(concatenated_df.index)


In [6]:
# Merge with fedfunds_df
concatenated_df = pd.merge(concatenated_df, fedfunds_df_monthly, left_index=True, right_index=True)


In [7]:
# Drop rows with NaN values
concatenated_df = concatenated_df.dropna()


In [8]:
# Shift target variable
concatenated_df['Target'] = concatenated_df['Close'].shift(-5)
concatenated_df = concatenated_df.dropna()

In [9]:
concatenated_df.head(10)

,Open,High,Low,Close,Adj Close,Volume,Fear_index,SPY_index,FEDFUNDS,Target
2014-04-30,14.9050,15.2280,14.9050,15.2065,15.2065,81772000.0,13.41,188.309998,0.09,14.6355
2014-05-01,15.2065,15.5240,15.2000,15.3945,15.3945,86572000.0,13.25,188.330002,0.09,14.4160
2014-05-02,15.5210,15.6645,15.2155,15.4005,15.4005,79902000.0,12.91,188.059998,0.09,14.6120
2014-05-05,15.3185,15.5115,15.2500,15.5025,15.5025,50398000.0,13.29,188.419998,0.09,15.1430
2014-05-06,15.4765,15.4905,14.8520,14.8690,14.8690,93646000.0,13.80,186.779999,0.09,15.2320
2014-05-07,14.7780,14.8200,14.3340,14.6355,14.6355,140304000.0,13.40,187.880005,0.09,14.8810
2014-05-08,14.5410,14.7940,14.3615,14.4160,14.4160,76964000.0,13.43,187.679993,0.09,14.7595
2014-05-09,14.5285,14.6840,14.2190,14.6120,14.6120,81260000.0,12.92,187.960007,0.09,14.8850
2014-05-12,14.7150,15.1670,14.7080,15.1430,15.1430,74592000.0,12.23,189.789993,0.09,14.8380
2014-05-13,15.1300,15.2800,15.0375,15.2320,15.2320,70198000.0,12.13,189.960007,0.09,15.0595


In [10]:
# Calculate technical indicators using finta
data = concatenated_df.copy()  # Use the existing DataFrame concatenated_df
data['MA'] = TA.SMA(data, 20)  # 20-period Simple Moving Average
data['RSI'] = TA.RSI(data, 14)  # 14-period RSI

# Calculate Bollinger Bands correctly
bb_bands = TA.BBANDS(data, 20, 2)

# Assign Bollinger Bands values to DataFrame columns
data['BB_UPPER'] = bb_bands['BB_UPPER']
data['BB_MIDDLE'] = bb_bands['BB_MIDDLE']
data['BB_LOWER'] = bb_bands['BB_LOWER']

# Convert index to datetime
data.index = pd.to_datetime(data.index)

# Display the calculated technical indicators
data.tail()


,Open,High,Low,Close,Adj Close,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
2024-03-18,175.800003,176.690002,174.279999,174.479996,174.479996,31250700.0,14.33,512.859985,5.33,179.710007,174.529500,53.840812,180.300610,174.529500,168.758390
2024-03-19,174.220001,176.089996,173.520004,175.899994,175.899994,26880900.0,13.82,515.710022,5.33,178.300003,174.970499,56.089506,179.574748,174.970499,170.366251
2024-03-20,176.139999,178.529999,174.639999,178.149994,178.149994,29947200.0,13.04,520.479980,5.33,179.830002,175.448499,59.459585,179.162610,175.448499,171.734389
2024-03-21,179.990005,181.419998,178.149994,178.149994,178.149994,32824300.0,12.92,522.200012,5.33,180.380005,175.626999,59.459585,179.504898,175.626999,171.749100
2024-03-22,177.750000,179.259995,176.750000,178.869995,178.869995,27964100.0,13.06,521.210022,5.33,180.970001,175.820998,60.582333,179.945113,175.820998,171.696884


In [11]:
# Define features and target
X = concatenated_df.drop("Close", axis=1)
y = concatenated_df["Close"]

In [12]:
data.drop(columns=['Open', 'High', 'Low', 'Close', 'Adj Close'], inplace=True)


In [13]:
# Display the modified DataFrame
data.head()

,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
2014-04-30,81772000.0,13.41,188.309998,0.09,14.6355,NaN,NaN,NaN,NaN,NaN
2014-05-01,86572000.0,13.25,188.330002,0.09,14.4160,NaN,100.000000,NaN,NaN,NaN
2014-05-02,79902000.0,12.91,188.059998,0.09,14.6120,NaN,100.000000,NaN,NaN,NaN
2014-05-05,50398000.0,13.29,188.419998,0.09,15.1430,NaN,100.000000,NaN,NaN,NaN
2014-05-06,93646000.0,13.80,186.779999,0.09,15.2320,NaN,28.329895,NaN,NaN,NaN


In [14]:
data_clean = data.dropna()
data_clean.index.rename('date', inplace=True)
data_clean.to_csv('../clean_data/AMZN_prepared_data.csv', index=True)

In [15]:
data_clean.head()

,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
date,,,,,,,,,,
2014-05-28,54520000.0,11.68,191.380005,0.09,15.339000,15.099575,59.851683,15.794230,15.099575,14.404920
2014-05-29,47310000.0,11.57,192.369995,0.09,16.178499,15.123700,63.620957,15.865879,15.123700,14.381521
2014-05-30,85204000.0,11.40,192.679993,0.09,16.483500,15.135350,61.507941,15.902327,15.135350,14.368373
2014-06-02,44068000.0,11.58,192.899994,0.10,16.375000,15.137425,55.518407,15.907640,15.137425,14.367210
2014-06-03,47584000.0,11.87,192.800003,0.10,16.620501,15.130275,53.044427,15.888787,15.130275,14.371763


In [16]:
# Define date cutoff for data split
date_cutoff = "2022-04-30"

# Split data
X_train = X[X.index <= date_cutoff]
X_test = X[X.index > date_cutoff]
y_train = y[y.index <= date_cutoff]
y_test = y[y.index > date_cutoff]

In [17]:
# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [18]:
# Define the number of folds
k = 10

# Initialize lists to store R-squared scores
train_r2_scores = []
test_r2_scores = []

# Initialize KFold
kf = KFold(n_splits=k, shuffle=True)

# Define the model architecture
model = Sequential([
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.6),
    Dense(units=8, activation='relu', kernel_regularizer=l2(0.0005)),
    Dropout(0.6),
    Dense(units=1)
])

# Example: Train with a smaller learning rate
from keras.optimizers import Adam
adam = Adam(learning_rate=0.0001)  # Adjust learning rate as needed
model.compile(optimizer=adam, loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Compile the model
model.compile(optimizer=adam, loss='mean_squared_error')

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_train_scaled):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    
    
    # Evaluate the model on training data
    train_predictions = model.predict(X_train_cv)
    train_r2 = r2_score(y_train_cv, train_predictions)
    train_r2_scores.append(train_r2)
    
    # Evaluate the model on test data
    test_predictions = model.predict(X_test_cv)
    test_r2 = r2_score(y_test_cv, test_predictions)
    test_r2_scores.append(test_r2)

# Calculate average R-squared scores
avg_train_r2 = np.mean(train_r2_scores)
avg_test_r2 = np.mean(test_r2_scores)

print("Average R-squared (Train):", avg_train_r2)
print("Average R-squared (Test):", avg_test_r2)

/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 574us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 341us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 333us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 537us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 535us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 655us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 429us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 338us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 569us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 330us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Average R-squared (Train): -2.298720481337436
Average R-squared (Test): -2.323420133283069


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize lists to store metrics for selected models
selected_train_r2 = []
selected_test_r2 = []
selected_train_mae = []
selected_train_mse = []
selected_test_mae = []
selected_test_mse = []

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_train_scaled):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    
    # Define a new model for each fold
    model_fold = Sequential([
        Dense(32, activation='relu', input_shape=(X_train_cv.shape[1],)),
        Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
        Dense(16, activation='relu'),
        Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
        Dense(1)  # Output layer
    ])
    
    # Compile the model with Adam optimizer and mean squared error loss
    model_fold.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    # Train the model
    model_fold.fit(X_train_cv, y_train_cv, epochs=50, batch_size=32, verbose=0)
    
    # Evaluate the model on training data
    train_predictions = model_fold.predict(X_train_cv)
    train_r2 = r2_score(y_train_cv, train_predictions)
    train_mae = mean_absolute_error(y_train_cv, train_predictions)
    train_mse = mean_squared_error(y_train_cv, train_predictions)
    
    # Evaluate the model on test data
    test_predictions = model_fold.predict(X_test_cv)
    test_r2 = r2_score(y_test_cv, test_predictions)
    test_mae = mean_absolute_error(y_test_cv, test_predictions)
    test_mse = mean_squared_error(y_test_cv, test_predictions)
    
    # Append metrics to the lists
    selected_train_r2.append(train_r2)
    selected_test_r2.append(test_r2)
    selected_train_mae.append(train_mae)
    selected_train_mse.append(train_mse)
    selected_test_mae.append(test_mae)
    selected_test_mse.append(test_mse)

# Print metrics for selected models
for idx, (train_r2, test_r2, train_mae, train_mse, test_mae, test_mse) in enumerate(zip(selected_train_r2, selected_test_r2, selected_train_mae, selected_train_mse, selected_test_mae, selected_test_mse), start=1):
    print(f"Model {idx} - Train R-squared: {train_r2}, Test R-squared: {test_r2}, Train MAE: {train_mae}, Train MSE: {train_mse}, Test MAE: {test_mae}, Test MSE: {test_mse}")


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 559us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 635us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 518us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 497us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 604us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 589us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 628us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 589us/step
Model 1 - Train R-squared: 0.958281745296925, Test R-squared: 0.9543962794794871, Train MAE: 9.480573977870282, Train MSE: 116.88156023136445, Test MAE: 9.992897512495588, Test MSE: 125.579136873779
Model 2 - Train R-squared: 0.9241069633996627, Test R-squared: 0.927279214745729, Train MAE: 12.649391745955262, Train MSE: 212.08687705009675, Test MAE: 12.343627727672615, Test MSE: 205.75098757805497
Model 3 - Train R-squared: 0.9205178069168307, Test R-squared: 0.9219358865444747, Train MAE: 13.152193339082123, Train MSE: 223.0255325204532, Test MAE: 12.807670257144135, Test MSE: 212.42172806829893
Model 4 - Train R-squared: 0.9361184560277527, Test R-squared: 0.9391788422673899, Train MAE: 11.709203585383877, Train MSE: 177.55459172620675, Test MAE: 11.96592585497503, Test MSE: 180.10801354152045
Model 5 - Train R-squared: 0.9189202643846903, Test R-squared: 0.9066575618150298, Train MAE: 12.894345790666204

In [20]:
# Filter models where both Train R-squared and Test R-squared are less than 0.96
filtered_indices = [i for i, (train_r2, test_r2) in enumerate(zip(selected_train_r2, selected_test_r2)) if train_r2 < 0.96 and test_r2 < 0.96]

# Calculate the absolute difference between train R-squared and test R-squared values for filtered models
abs_diff_r2_filtered = np.abs(np.array(selected_train_r2)[filtered_indices] - np.array(selected_test_r2)[filtered_indices])

# Find the index of the model with the smallest absolute difference among filtered models
best_model_index = filtered_indices[np.argmin(abs_diff_r2_filtered)]

# Retrieve the metrics for the best model
best_train_r2 = selected_train_r2[best_model_index]
best_test_r2 = selected_test_r2[best_model_index]
best_train_mae = selected_train_mae[best_model_index]
best_train_mse = selected_train_mse[best_model_index]
best_test_mae = selected_test_mae[best_model_index]
best_test_mse = selected_test_mse[best_model_index]

# Print metrics for the best model
print(f"Best Model - Train R-squared: {best_train_r2}, Test R-squared: {best_test_r2}, Train MAE: {best_train_mae}, Train MSE: {best_train_mse}, Test MAE: {best_test_mae}, Test MSE: {best_test_mse}")


Best Model - Train R-squared: 0.9205178069168307, Test R-squared: 0.9219358865444747, Train MAE: 13.152193339082123, Train MSE: 223.0255325204532, Test MAE: 12.807670257144135, Test MSE: 212.42172806829893


In [21]:
# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the model architecture
model = Sequential([
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_scaled.shape[1],)),
    Dropout(0.6),
    Dense(units=8, activation='relu', kernel_regularizer=l2(0.0005)),
    Dropout(0.6),
    Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on the entire dataset
model.fit(X_scaled, y, epochs=50, batch_size=32, verbose=0)

# Predict sequentially on each data point
all_predictions = model.predict(X_scaled)

# Ensure the number of predictions matches the original dataset
assert len(all_predictions) == len(X_scaled)

# Create a DataFrame to store the actual and predicted values
predictions_df = pd.DataFrame({'Actual': y, 'Predicted': all_predictions.flatten()}, index=X.index)

# Ensure index uniqueness in both the original dataset and predictions DataFrame
data_clean_unique_index = data_clean.index.drop_duplicates()
predictions_df = predictions_df.loc[data_clean_unique_index]

# Display the DataFrame
predictions_df


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 513us/step


,Actual,Predicted
date,,
2014-05-28,15.508000,19.057434
2014-05-29,15.689000,19.187449
2014-05-30,15.627500,18.891146
2014-06-02,15.442000,19.188818
2014-06-03,15.359500,19.116161
...,...,...
2024-03-18,174.479996,130.179779
2024-03-19,175.899994,129.415680
2024-03-20,178.149994,130.645569


In [22]:
predictions_df.to_csv('../predicted_data/AMZN_predicted_data.csv', index=True)

In [23]:
# Check for NaN values in the predictions DataFrame
nan_values = predictions_df.isnull().sum().sum()

if nan_values == 0:
    print("No NaN values found in the predictions DataFrame.")
    print(predictions_df)
else:
    print(f"Found {nan_values} NaN values in the predictions DataFrame. Please check your data or model.")


No NaN values found in the predictions DataFrame.
                Actual   Predicted
date                              
2014-05-28   15.508000   19.057434
2014-05-29   15.689000   19.187449
2014-05-30   15.627500   18.891146
2014-06-02   15.442000   19.188818
2014-06-03   15.359500   19.116161
...                ...         ...
2024-03-18  174.479996  130.179779
2024-03-19  175.899994  129.415680
2024-03-20  178.149994  130.645569
2024-03-21  178.149994  132.445633
2024-03-22  178.869995  131.743652

[2473 rows x 2 columns]


In [24]:
# Calculate the percentage difference between actual and predicted values
predictions_df['Percentage Difference (%)'] = ((predictions_df['Predicted'] - predictions_df['Actual']) / predictions_df['Actual']) * 100

# Display the DataFrame with percentage difference
predictions_df


,Actual,Predicted,Percentage Difference (%)
date,,,
2014-05-28,15.508000,19.057434,22.887762
2014-05-29,15.689000,19.187449,22.298735
2014-05-30,15.627500,18.891146,20.883991
2014-06-02,15.442000,19.188818,24.263813
2014-06-03,15.359500,19.116161,24.458227
...,...,...,...
2024-03-18,174.479996,130.179779,-25.389854
2024-03-19,175.899994,129.415680,-26.426558
2024-03-20,178.149994,130.645569,-26.665409


In [25]:
# Calculate the absolute percentage difference for each data point
predictions_df['Abs_Percentage_Diff'] = abs((predictions_df['Actual'] - predictions_df['Predicted']) / predictions_df['Actual']) * 100

# Calculate the average percentage difference
avg_percentage_diff = predictions_df['Abs_Percentage_Diff'].mean()

print("Average Percentage Difference (%):", avg_percentage_diff)


Average Percentage Difference (%): 28.50154982368399
